In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
import json
import ast

In [2]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [3]:
from statsmodels.stats import diagnostic
import statsmodels.formula.api as smf
import statsmodels.api as sm

/home/leo/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/leo/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [4]:
df = pd.read_csv('./data/movie_features.csv')

In [5]:
df.head()

,movie_id,title,budget,revenue,release_date,runtime,original_language,genres,number_characters,below18,above60,male_ratio,average_actor_exp,multiplier,year,year_group,category
0,951491.0,Saw X,13000000.0,100900000.0,2023-09-26,118.0,en,"['Horror', 'Thriller']",7.0,0.000,0.142857,0.571429,5.857143,7.761538,2023,2020,low
1,678512.0,Sound of Freedom,14500000.0,247800000.0,2023-07-03,131.0,en,"['Action', 'Drama']",7.0,0.000,0.142857,0.857143,7.285714,17.089655,2023,2020,low
2,820525.0,After Everything,14000000.0,9700000.0,2023-09-13,93.0,en,"['Romance', 'Drama']",10.0,0.000,0.000000,0.500000,3.200000,0.692857,2023,2020,low
3,1008042.0,Talk to Me,4500000.0,90600000.0,2023-07-26,95.0,en,"['Horror', 'Thriller']",8.0,0.125,0.125000,0.500000,1.250000,20.133333,2023,2020,low
4,1058694.0,Radical,10000000.0,16100000.0,2023-10-19,127.0,en,['Drama'],1.0,0.000,1.000000,1.000000,9.000000,1.610000,2023,2020,low


In [6]:
df['normalized_runtime'] = (df['runtime'] - df['runtime'].mean())/df['runtime'].std()

In [7]:
mod_1 = smf.ols(formula='multiplier ~ normalized_runtime', data=df)

In [8]:
np.random.seed(2)
res = mod_1.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             multiplier   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     4.489
Date:                Fri, 08 Dec 2023   Prob (F-statistic):             0.0343
Time:                        15:24:55   Log-Likelihood:                -7046.8
No. Observations:                1701   AIC:                         1.410e+04
Df Residuals:                    1699   BIC:                         1.411e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              5.2151      0

In [9]:
# Performing one-hot encoding for the 'track' column
original_language_onehot = pd.get_dummies(df['original_language'], prefix='onehot', prefix_sep='_')

# Concatenate the new one-hot encoded columns to the original dataframe
df = pd.concat([df, original_language_onehot], axis=1)

for col in df.columns:
    print(col)

movie_id
title
budget
revenue
release_date
runtime
original_language
genres
number_characters
below18
above60
male_ratio
average_actor_exp
multiplier
year
year_group
category
normalized_runtime
onehot_cn
onehot_cs
onehot_da
onehot_de
onehot_en
onehot_es
onehot_fi
onehot_fr
onehot_hi
onehot_id
onehot_it
onehot_ja
onehot_kn
onehot_ko
onehot_la
onehot_no
onehot_pt
onehot_ru
onehot_sv
onehot_te
onehot_th
onehot_tr
onehot_zh


In [27]:
# Select all the onehot feature names previously created
# selected_cols = [col for col in df.columns if 'onehot' in col]

grouped_by_language_df = df.groupby(['original_language']).size()
print(grouped_by_language_df)

original_language
cn      10
cs       1
da       5
de       4
en    1509
es      21
fi       1
fr      34
hi      24
id       2
it       5
ja      22
kn       1
ko      20
la       1
no       4
pt       1
ru      11
sv       3
te       2
th       2
tr       1
zh      17
dtype: int64


In [31]:
# Selecting only languages above a certain threshold
mod_2 = sm.OLS(df['multiplier'], df[['onehot_cn', 'onehot_en', 'onehot_es', 'onehot_fr', 'onehot_hi', 'onehot_ja', 'onehot_ko', 'onehot_ru', 'onehot_zh']])

In [32]:
np.random.seed(2)
res = mod_2.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.138
Model:                            OLS   Adj. R-squared (uncentered):              0.134
Method:                 Least Squares   F-statistic:                              30.19
Date:                Fri, 08 Dec 2023   Prob (F-statistic):                    2.92e-49
Time:                        15:57:32   Log-Likelihood:                         -7016.3
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1692   BIC:                                  1.410e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [33]:
# Trying to improve it by removing non-statistically significant ones
mod_3 = sm.OLS(df['multiplier'], df[['onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])

In [34]:
np.random.seed(2)
res = mod_3.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.133
Model:                            OLS   Adj. R-squared (uncentered):              0.131
Method:                 Least Squares   F-statistic:                              64.95
Date:                Fri, 08 Dec 2023   Prob (F-statistic):                    3.66e-51
Time:                        15:58:47   Log-Likelihood:                         -7021.9
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1697   BIC:                                  1.407e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [35]:
# Trying to combine with normalized runtime
mod_4 = sm.OLS(df['multiplier'], df[['normalized_runtime', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_4.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.134
Model:                            OLS   Adj. R-squared (uncentered):              0.131
Method:                 Least Squares   F-statistic:                              52.36
Date:                Fri, 08 Dec 2023   Prob (F-statistic):                    1.25e-50
Time:                        16:00:16   Log-Likelihood:                         -7020.9
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1696   BIC:                                  1.408e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [38]:
# Non sense to add runtime, trying number of characters
mod_5 = sm.OLS(df['multiplier'], df[['number_characters', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_5.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.135
Model:                            OLS   Adj. R-squared (uncentered):              0.133
Method:                 Least Squares   F-statistic:                              53.04
Date:                Fri, 08 Dec 2023   Prob (F-statistic):                    2.90e-51
Time:                        16:02:43   Log-Likelihood:                         -7019.4
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1696   BIC:                                  1.408e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [40]:
# Non sense to add runtime, trying to add other features
mod_6 = sm.OLS(df['multiplier'], df[['below18', 'above60', 'male_ratio', 'average_actor_exp', 'number_characters', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_6.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.139
Model:                            OLS   Adj. R-squared (uncentered):              0.134
Method:                 Least Squares   F-statistic:                              30.35
Date:                Fri, 08 Dec 2023   Prob (F-statistic):                    1.61e-49
Time:                        16:08:00   Log-Likelihood:                         -7015.7
No. Observations:                1701   AIC:                                  1.405e+04
Df Residuals:                    1692   BIC:                                  1.410e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [42]:
# Non sense to add runtime, trying to add other features
mod_7 = sm.OLS(df['multiplier'], df[['average_actor_exp', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_7.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:             multiplier   R-squared (uncentered):                   0.138
Model:                            OLS   Adj. R-squared (uncentered):              0.135
Method:                 Least Squares   F-statistic:                              54.25
Date:                Fri, 08 Dec 2023   Prob (F-statistic):                    2.20e-52
Time:                        16:08:50   Log-Likelihood:                         -7016.8
No. Observations:                1701   AIC:                                  1.404e+04
Df Residuals:                    1696   BIC:                                  1.407e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [43]:
# Convert lists of genres to strings (if not already)
df['genres_as_string'] = df['genres'].astype(str)

# Use get_dummies to create dummy variables for each genre
dummy_genres = df['genres_as_string'].str.get_dummies(sep=', ')

# Concatenate the dummy variables with the original DataFrame
df = pd.concat([df, dummy_genres], axis=1)

# Display the modified DataFrame
print(df)

       movie_id             title      budget      revenue release_date  \
0      951491.0             Saw X  13000000.0  100900000.0   2023-09-26   
1      678512.0  Sound of Freedom  14500000.0  247800000.0   2023-07-03   
2      820525.0  After Everything  14000000.0    9700000.0   2023-09-13   
3     1008042.0        Talk to Me   4500000.0   90600000.0   2023-07-26   
4     1058694.0           Radical  10000000.0   16100000.0   2023-10-19   
...         ...               ...         ...          ...          ...   
1696    11644.0          Blow Out  18000000.0   12000000.0   1981-07-24   
1697      849.0             Krull  27000000.0   16519460.0   1983-07-29   
1698      848.0      Dragonslayer  18000000.0   14110013.0   1981-06-26   
1699    10724.0           Firefox  18000000.0   70687344.0   1982-06-18   
1700    11393.0       The Natural  28000000.0   47951979.0   1984-05-11   

      runtime original_language  \
0       118.0                en   
1       131.0                

In [ ]:
# Non sense to add runtime, trying to add other features
mod_7 = sm.OLS(df['multiplier'], df[['Science Fiction', 'average_actor_exp', 'onehot_cn', 'onehot_en', 'onehot_ja', 'onehot_ko']])
np.random.seed(2)
res = mod_7.fit()
print(res.summary())